In [1]:
import pandas as pd

In [15]:
# PARTICIPANT_DATA_WITH_DATES = "/cs/labs/michall/roeizucker/ukb_files/diagnosis.csv"
PARTICIPANT_DATA_WITH_DATES = "/cs/usr/roeizucker/my_storage/IIH/updated_phenotypes/diagnosis_ethnicity_25_11_24_participant_participant.csv"

# requested_phenotypes_list = [(["H47.1 Papilloedema, unspecified"],"updated_PAP","B"),
#                              (["G93.2 Benign intracranial hypertension"],"updated_IIH","B"),
#                             (["H47.1 Papilloedema, unspecified","G93.2 Benign intracranial hypertension"],"updated_Both","B"),
                             
#                             (["H47.1 Papilloedema, unspecified"],"updated_PAP_F","Female"),
#                             (["G93.2 Benign intracranial hypertension"],"updated_IIH_F","Female"),
#                             (["H47.1 Papilloedema, unspecified","G93.2 Benign intracranial hypertension"],"updated_Both_F","Female"),
                             
#                             (["H47.1 Papilloedema, unspecified"],"updated_PAP_M","Male"),
#                             (["G93.2 Benign intracranial hypertension"],"updated_IIH_M","Male"),
#                             (["H47.1 Papilloedema, unspecified","G93.2 Benign intracranial hypertension"],"updated_Both_M","Male")]
# requested_phenotypes_list = [
#     (["L63","L64","L65"],"L63-5","B"),
#     (["L63","L64"],"L63-4","B"),
#     (["L63"],"L63","B"),
#     (["L65"],"L65","B")
# ]
requested_phenotypes_list = [
    (["H47.1"],"H47.1","B"),
    (["G93.2"],"G93.2","B"),
    (["H47.1","G93.2"],"both_phen","B"),
]
# PAP,IIH,both,PAP_F,IIH_F,both_F,PAP_M,IIH_M,both_M
OLD_FILE = "/cs/labs/michall/roeizucker/IIH/PWAS/ukbb_dataset.csv"
RESULT_PATH = "~/my_storage/IIH/updated_phenotypes/ukbb_dataset_updated_11.csv"


In [3]:
DIAGNOSES_FIELD_NAME = "Diagnoses - ICD10"
# REQUIRED_FILEDS = ['Participant ID','Sex','Year of birth(participant - p34)',DIAGNOSES_FIELD_NAME]
REQUIRED_FILEDS = ['Participant ID','Sex','Year of birth',DIAGNOSES_FIELD_NAME]
SEPERETOR = "|"

In [4]:
old_df = pd.read_csv(OLD_FILE)

In [5]:
df = pd.read_csv(PARTICIPANT_DATA_WITH_DATES,low_memory=False,usecols=REQUIRED_FILEDS)

In [6]:
def find_phen(value,requested_diags,sex="B"):

    if not isinstance(value[DIAGNOSES_FIELD_NAME], str):
        return None
    diags = (value[DIAGNOSES_FIELD_NAME].strip('][').split(SEPERETOR))
    new_diags = []
    for diag in diags:
        diag = diag.replace("\"","")
        new_diags.append(diag)
    for req_diag in requested_diags:
        for participant_diag in new_diags:
            if req_diag in participant_diag and (sex=="B" or sex==value["Sex"]):
                return 1
    if sex=="B" or sex==value["Sex"]:
        return 0
    else:
        return None
    
    


In [7]:
!head -n 2 /cs/labs/michall/roeizucker/IIH/PWAS/ukbb_dataset_updated.csv

sample_index,eid,PAP,IIH,both,PAP_F,IIH_F,both_F,PAP_M,IIH_M,both_M,const,sex,year_of_birth,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35,PC36,PC37,PC38,PC39,PC40,AC_stockport_pilot,AC_manchester,AC_oxford,AC_cardiff,AC_glasgow,AC_edinburgh,AC_stoke,AC_reading,AC_bury,AC_newcastle,AC_bristol,AC_barts,AC_nottingham,AC_sheffield,AC_liverpool,AC_middlesborough,AC_hounslow,AC_croydon,AC_birmingham,AC_swansea,AC_wrexham,AC_cheadle_revisit,AC_cheadle_imaging,AC_reading_imaging,AC_newcastle_imaging,batch_Batch_b001,batch_Batch_b002,batch_Batch_b003,batch_Batch_b004,batch_Batch_b005,batch_Batch_b006,batch_Batch_b007,batch_Batch_b008,batch_Batch_b009,batch_Batch_b010,batch_Batch_b011,batch_Batch_b012,batch_Batch_b013,batch_Batch_b014,batch_Batch_b015,batch_Batch_b016,batch_Batch_b017,batch_Batch_b018,batch_Batch_b019,batch_Batch_b020,batch_Batch_b021,batch_Batch_b022,batch_Bat

In [8]:
for phen in requested_phenotypes_list:
    df[phen[1]] = df.apply(lambda x: find_phen(x,phen[0],phen[2]),axis=1)
df["eid"] = df["Participant ID"]

In [9]:
small_df = df[["eid"] + list(map(lambda x:x[1],requested_phenotypes_list))]

In [10]:
updated_df = pd.merge(old_df,small_df,on="eid",how="left")

In [13]:
df[df["eid"] == 2745120]

,Participant ID,Diagnoses - ICD10,Sex,Year of birth,H47.1,G93.2,both_phen,eid
401708,2745120,A09.9 Gastroenteritis and colitis of unspecifi...,Male,1945,0.0,0.0,0.0,2745120


In [14]:
RESULT_PATH

'roeizucker@chaperone-02:~/my_storage/IIH/updated_phenotypes/ukbb_dataset_updated_11.csv'

In [20]:
updated_df.to_csv(RESULT_PATH,index=False)

In [ ]:
updated_df[(updated_df["both"] ==1 )& (updated_df["updated_Both"] ==0)][["eid","updated_Both","both"]]

In [ ]:
list(map(lambda x:x[1],requested_phenotypes_list))

In [18]:
RESULT_PATH

'~/my_storage/IIH/updated_phenotypes/ukbb_dataset_updated_11.csv'

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer